# Installing required packages - beautifulsoup4

In [121]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests

# Get a local copy of the Wikipedia article

In [131]:
import requests
wiki_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

from bs4 import BeautifulSoup
soup = BeautifulSoup(wiki_url,"lxml")
#print(soup.prettify())

tables = soup.find("table",{"class":"wikitable sortable"})

In [132]:
rows = Canada_table.findAll("tr")
row_lengths = [len(r.findAll(['th', 'td'])) for r in rows]
ncols = max(row_lengths)
nrows = len(rows)

# Span the rows and columns of wiki table and convert to a list

In [145]:
    # preinit list of lists
    rows = tables.findAll("tr")
    row_lengths = [len(r.findAll(['th', 'td'])) for r in rows]
    ncols = max(row_lengths)
    nrows = len(rows)
    data = []
    for i in range(nrows):
        rowD = []
        for j in range(ncols):
            rowD.append('')
        data.append(rowD)

    # process html
    for i in range(len(rows)):
        row = rows[i]
        rowD = []
        cells = row.findAll(["td", "th"])
        for j in range(len(cells)):
            cell = cells[j]

            #lots of cells span cols and rows so lets deal with that
            cspan = int(cell.get('colspan', 1))
            rspan = int(cell.get('rowspan', 1))
            l = 0
            for k in range(rspan):
                # Shifts to the first empty cell of this row
                while data[i + k][j + l]:
                    l += 1
                for m in range(cspan):
                    cell_n = j + l + m
                    row_n = i + k
                    # in some cases the colspan can overflow the table, in those cases just get the last item
                    cell_n = min(cell_n, len(data[row_n])-1)
                    data[row_n][cell_n] += cell.text
                    #print(cell.text)

        data.append(rowD)

# Convert the list to a dataframe

In [196]:
import pandas as pd
import numpy as np
df = pd.DataFrame(data,columns=data[0])

df1 = df.drop([0])

df2 = df1[:287]

df3 = df2.rename(columns = {"Postcode": "PostalCode","Neighbourhood\n": "Neighbourhood"})
df3["Neighbourhood"] = df3["Neighbourhood"].str.replace(r'\n$', '')

df4 = df3[df3.Borough != "Not assigned"]
#df4[df4.PostalCode == "M6A"]
#df4[df4.Neighbourhood == "Not assigned"]
df4["Neighbourhood"] = np.where(df4["Neighbourhood"] == "Not assigned", df4["Borough"], df4["Neighbourhood"])

df5 = df4.groupby(["PostalCode","Borough"])["Neighbourhood"].apply(', '.join).reset_index()

df5

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [197]:
df5.shape

(103, 3)